In [4]:
import h5py
import numpy as np
import pandas as pd
from scipy.signal import spectrogram
import librosa
from tqdm import tqdm

In [ ]:
# Parameters for spectrogram
FS = 100
NPERSEG = 64
NOVERLAP = 48

def compute_spectrograms(waveforms, fs=FS, nperseg=NPERSEG, noverlap=NOVERLAP):
    """
    Compute three-channel spectrogram and log-normalized spectrogram.
    """
    specs = []
    for i in range(3):
        f, t, Sxx = spectrogram(waveforms[i], fs=fs, nperseg=nperseg, noverlap=noverlap)
        specs.append(Sxx)
    specs = np.stack(specs, axis=0)  # (3, F, T)
    # Log normalization
    log_specs = np.log1p(specs)
    return specs, log_specs

def extract_features(waveforms, fs=FS):
    """
    Extract additional features to help classify pre- vs post-events.

    Features:
    - Max and RMS amplitude per channel
    - Spectral centroid and bandwidth (mean over time)
    - Zero-crossing rate (mean)
    - Energy ratio in a specific frequency band (e.g., 5–10 Hz)
    """
    features = {}

    # Basic amplitude features
    for i in range(3):
        ch_data = waveforms[i]
        features[f'max_amp_ch{i}'] = np.max(np.abs(ch_data))
        features[f'rms_amp_ch{i}'] = np.sqrt(np.mean(ch_data**2))

    # Advanced spectral and temporal features using librosa
    # For each channel, compute spectral centroid, bandwidth, and zero-crossing rate
    for i in range(3):
        ch_data = waveforms[i]

        # Spectral centroid & bandwidth
        centroid = librosa.feature.spectral_centroid(y=ch_data, sr=fs)
        bandwidth = librosa.feature.spectral_bandwidth(y=ch_data, sr=fs)
        # Take mean over time frames
        features[f'spectral_centroid_ch{i}'] = np.mean(centroid)
        features[f'spectral_bandwidth_ch{i}'] = np.mean(bandwidth)

        # Zero-crossing rate
        zcr = librosa.feature.zero_crossing_rate(y=ch_data)
        features[f'zcr_ch{i}'] = np.mean(zcr)

        # Energy ratio in a band (e.g., 5–10 Hz)
        # Compute FFT
        fft_vals = np.fft.rfft(ch_data)
        freqs = np.fft.rfftfreq(len(ch_data), 1/fs)
        
        # Total energy
        total_energy = np.sum(np.abs(fft_vals)**2)
        
        # Frequency band energy: 5–10 Hz
        band_mask = (freqs >= 5) & (freqs <= 10)
        band_energy = np.sum(np.abs(fft_vals[band_mask])**2)
        
        # Ratio of band energy to total energy
        band_ratio = band_energy / (total_energy + 1e-12)  # Avoid division by zero
        features[f'band_5_10_energy_ratio_ch{i}'] = band_ratio

    return features

# Files
pre_file = r"C:\Users\frmar\OneDrive\Desktop\EQML Project\data\dataset\NRCA\NRCA_waveforms_pre.hdf5"
post_file = r"C:\Users\frmar\OneDrive\Desktop\EQML Project\data\dataset\NRCA\NRCA_waveforms_post.hdf5"

data_rows = []

# Process pre file
with h5py.File(pre_file, 'r') as hdf:
    for event_id in tqdm(hdf.keys()):
        waveforms = hdf[event_id][()]  # (3, 2500)
        try:
            # Compute spectrograms
            specs, log_specs = compute_spectrograms(waveforms)
            
            # Extract features
            feat = extract_features(waveforms)
            
            # Create row dict
            row = {
                'ID': event_id,
                'label': 'pre',
                'spectrograms': specs,        # (3, F, T)
                'log_spectrograms': log_specs # (3, F, T)
            }
            # Add extracted features to row
            row.update(feat)
            
            data_rows.append(row)

        except Exception as e:
            print(f"Error {e} processing event {event_id}")

# Process post file
with h5py.File(post_file, 'r') as hdf:
    for event_id in tqdm(hdf.keys()):
        try:
            waveforms = hdf[event_id][()]  # (3, 2500)
            
            # Compute spectrograms
            specs, log_specs = compute_spectrograms(waveforms)
            
            # Extract features
            feat = extract_features(waveforms)
            
            # Create row dict
            row = {
                'ID': event_id,
                'label': 'post',
                'spectrograms': specs,
                'log_spectrograms': log_specs
            }
            row.update(feat)
            
            data_rows.append(row)
        except Exception as e:
            print(f"Error {e} processing event {event_id}")
# Create DataFrame
df = pd.DataFrame(data_rows)

print(df.head())

df.to_pickle("dataframe_with_spectrograms.pkl")

  8%|▊         | 347/4599 [00:02<00:25, 165.64it/s]

Error index 1 is out of bounds for axis 0 with size 1 processing event NRCA.IV.100019941_EV
Error index 1 is out of bounds for axis 0 with size 1 processing event NRCA.IV.100019946_EV
Error index 1 is out of bounds for axis 0 with size 1 processing event NRCA.IV.100019950_EV


 70%|██████▉   | 3208/4599 [00:21<00:08, 155.77it/s]

Error index 2 is out of bounds for axis 0 with size 2 processing event NRCA.IV.100709001_EV


100%|██████████| 4599/4599 [00:30<00:00, 150.91it/s]


                     ID label  \
0  NRCA.IV.100000017_EV   pre   
1  NRCA.IV.100000034_EV   pre   
2  NRCA.IV.100000038_EV   pre   
3  NRCA.IV.100000096_EV   pre   
4  NRCA.IV.100000170_EV   pre   

                                        spectrograms  \
0  [[[0.6092216618208547, 0.09465900931437798, 2....   
1  [[[8.684206150031395, 25.721654569982736, 0.62...   
2  [[[0.04428579181868711, 2.2685307762803806, 5....   
3  [[[1.1787911839823475, 2.8593425591146877, 5.5...   
4  [[[0.0797510528282796, 0.24318097433061106, 0....   

                                    log_spectrograms   max_amp_ch0  \
0  [[[0.4757506222388221, 0.09044290775239705, 1....  78022.982454   
1  [[[2.270496326543511, 3.2854742692806185, 0.48...   1788.026363   
2  [[[0.043333198951351896, 1.184340580052133, 1....   1661.176790   
3  [[[0.7787702202470957, 1.3504968474975114, 1.8...  31699.049927   
4  [[[0.07673050792490758, 0.21767339672602326, 0...   3557.065390   

   rms_amp_ch0   max_amp_ch1  rms_amp_ch1  